**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Linux namespaces

[ ![](namespaces.png) ](https://8gwifi.org/docs/linux-namespace.jsp)

![container vs hypervisor](docker-hypervisor.png)

In [1]:
docker ps

CONTAINER ID   IMAGE                  COMMAND                  CREATED          STATUS          PORTS     NAMES
5da141e6ccb8   nginx                  "/docker-entrypoint.…"   32 minutes ago   Up 32 minutes             k8s_my-nginx_my-nginx-5b56ccd65f-s8qwn_default_af41adda-6f71-4e0e-9ed3-2b066c044bf7_0
3f53be8b44df   nginx                  "/docker-entrypoint.…"   32 minutes ago   Up 32 minutes             k8s_my-nginx_my-nginx-5b56ccd65f-8krqn_default_7a69034a-66a1-49a2-9847-f74202b4a1cb_0
49d80ef0b84c   k8s.gcr.io/pause:3.2   "/pause"                 32 minutes ago   Up 32 minutes             k8s_POD_my-nginx-5b56ccd65f-8krqn_default_7a69034a-66a1-49a2-9847-f74202b4a1cb_0
e66281d22c21   k8s.gcr.io/pause:3.2   "/pause"                 32 minutes ago   Up 32 minutes             k8s_POD_my-nginx-5b56ccd65f-s8qwn_default_af41adda-6f71-4e0e-9ed3-2b066c044bf7_0
267e2081d55b   85069258b98a           "/storage-provisioner"   2 hours ago      Up 2 hours                k8s_storage-provisioner_

In [2]:
crictl ps

CONTAINER           IMAGE                                                                           CREATED             STATE               NAME                      ATTEMPT             POD ID
5da141e6ccb8b       nginx@sha256:10b8cc432d56da8b61b070f4c7d2543a9ed17c2b23010b43af434fd40e2ca4aa   32 minutes ago      Running             my-nginx                  0                   e66281d22c216
3f53be8b44dfe       nginx@sha256:10b8cc432d56da8b61b070f4c7d2543a9ed17c2b23010b43af434fd40e2ca4aa   32 minutes ago      Running             my-nginx                  0                   49d80ef0b84cf
267e2081d55b0       85069258b98ac                                                                   2 hours ago         Running             storage-provisioner       0                   1acba0cf04de9
c42428440d943       bfe3a36ebd252                                                                   2 hours ago         Running             coredns                   0                   37a3f07e57b5b
9a12ffc

In [3]:
POD=k8s_my-nginx_my-nginx-5b56ccd65f-s8qwn_default_af41adda-6f71-4e0e-9ed3-2b066c044bf7_0

In [4]:
STATE_PID=$(docker inspect --format '{{.State.Pid}}' $POD)
echo Enter
echo nsenter -t $STATE_PID -n bash
echo in a terminal  

Enter
nsenter -t 23320 -n bash
in a terminal


In [5]:
nsenter --help


Usage:
 nsenter [options] [<program> [<argument>...]]

Run a program with namespaces of other processes.

Options:
 -a, --all              enter all namespaces
 -t, --target <pid>     target process to get namespaces from
 -m, --mount[=<file>]   enter mount namespace
 -u, --uts[=<file>]     enter UTS namespace (hostname etc)
 -i, --ipc[=<file>]     enter System V IPC namespace
 -n, --net[=<file>]     enter network namespace
 -p, --pid[=<file>]     enter pid namespace
 -C, --cgroup[=<file>]  enter cgroup namespace
 -U, --user[=<file>]    enter user namespace
 -S, --setuid <uid>     set uid in entered namespace
 -G, --setgid <gid>     set gid in entered namespace
     --preserve-credentials do not touch uids or gids
 -r, --root[=<dir>]     set the root directory
 -w, --wd[=<dir>]       set the working directory
 -F, --no-fork          do not fork before exec'ing <program>
 -Z, --follow-context   set SELinux context according to --target PID

 -h, --help             display this help
 -V

In [6]:
nsenter -t $STATE_PID -n ip a

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
2: tunl0@NONE: <NOARP> mtu 1480 qdisc noop state DOWN group default qlen 1000
    link/ipip 0.0.0.0 brd 0.0.0.0
6: eth0@if7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:ac:11:00:03 brd ff:ff:ff:ff:ff:ff link-netnsid 0
    inet 172.17.0.3/16 brd 172.17.255.255 scope global eth0
       valid_lft forever preferred_lft forever


In [7]:
ip a

256: eth0@if257: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:c0:a8:31:02 brd ff:ff:ff:ff:ff:ff link-netnsid 0
    inet 192.168.49.2/24 brd 192.168.49.255 scope global eth0
       valid_lft forever preferred_lft forever
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
2: tunl0@NONE: <NOARP> mtu 1480 qdisc noop state DOWN group default qlen 1000
    link/ipip 0.0.0.0 brd 0.0.0.0
3: docker0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:db:c5:40:fb brd ff:ff:ff:ff:ff:ff
    inet 172.17.0.1/16 brd 172.17.255.255 scope global docker0
       valid_lft forever preferred_lft forever
5: veth57edace@if4: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue master docker0 state UP group default 
    l

 # Danger!
 
 OpenVpn works and could be used as backdoor

In [8]:
nsenter -t $STATE_PID -a hostname

my-nginx-5b56ccd65f-s8qwn


In [9]:
nsenter -t $STATE_PID -a ip a

nsenter: failed to execute ip: No such file or directory


: 127

In [10]:
nsenter -t $STATE_PID -n ip route

default via 172.17.0.1 dev eth0 
172.17.0.0/16 dev eth0 proto kernel scope link src 172.17.0.3 


In [11]:
ip route

default via 192.168.49.1 dev eth0 
172.17.0.0/16 dev docker0 proto kernel scope link src 172.17.0.1 
192.168.49.0/24 dev eth0 proto kernel scope link src 192.168.49.2 


In [12]:
nsenter -t $STATE_PID -a ls

bin   docker-entrypoint.d   home   media  proc	sbin  tmp
boot  docker-entrypoint.sh  lib    mnt	  root	srv   usr
dev   etc		    lib64  opt	  run	sys   var


In [13]:
ls

 AccountWithRBAC.ipynb       Overview.ipynb
 AuditLogs.ipynb             PodSecurityPolicyBasic.ipynb
 ClusterOverview.ipynb       PodSecurityPolicyPermissive.ipynb
 ContainerdCloudbomb.ipynb   PodSecurityPolicyStrict.ipynb
 CrioBomb.ipynb             'Postgres Zalando.ipynb'
 Helm.ipynb                  ServiceAccountToken.ipynb
 Ingress.ipynb               Unprivileged.ipynb
 IngressStart.ipynb          audit-policy.yaml
 Istio-Traffic-Mgmt.ipynb    cloudbomb
 Istio1.8.2.ipynb            docker-hypervisor.png
 IstioHack.ipynb             istio-1.8.2
 Kube-Scan-Cloudbomb.ipynb   kube-bench.ipynb
 KubernetesBasics.ipynb      notebooks.tgz
'Linux Namespaces.ipynb'     openssl
 NetworkPolicy.ipynb         postgres-operator
 OpenPolicyAgent.ipynb


In [14]:
nsenter -t $STATE_PID -a ps auxwww

nsenter: failed to execute ps: No such file or directory


: 127

In [15]:
ps auxww

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.0  22420  7964 ?        Ss   08:15   0:01 /sbin/init
root         135  0.0  0.0  29268  8544 ?        S<s  08:15   0:00 /lib/systemd/systemd-journald
root         155  0.0  0.2 1563460 44752 ?       Ssl  08:15   0:04 /usr/bin/containerd
root         159  0.0  0.0   4384  1172 pts/0    Ss+  08:15   0:00 /sbin/agetty -o -p -- \u --noclear --keep-baud console 115200,38400,9600 xterm
root         160  0.0  0.0   2860  1068 tty2     Ss+  08:15   0:00 /sbin/agetty -o -p -- \u --noclear tty2 linux
root         161  0.0  0.0   2860  1040 tty3     Ss+  08:15   0:00 /sbin/agetty -o -p -- \u --noclear tty3 linux
root         162  0.0  0.0   2860  1012 tty4     Ss+  08:15   0:00 /sbin/agetty -o -p -- \u --noclear tty4 linux
root         163  0.0  0.0   2860  1016 tty5     Ss+  08:15   0:00 /sbin/agetty -o -p -- \u --noclear tty5 linux
root         168  0.0  0.0   2860  1156 tty6     Ss+  08:15   0:0